In [1]:
import numpy as np
import matplotlib.pyplot as plt 
import pandas as pd

In [2]:
from sqlite3 import connect

## Opening the databases

In [3]:
def qry(q, connection = connect("database.sqlite")):
    df = pd.read_sql_query(q, connection)
    connection.close
    return df

The database consists of 4 tables: users, method, grade, ascent

In [4]:
tables = qry("SELECT name FROM sqlite_master")
tables

,name
0,user
1,method
2,grade
3,ascent


In [5]:
%%time
df_user = qry("SELECT * FROM USER")# 62'593 users x 22 columns
df_grade = qry("SELECT * FROM grade")# 83 grades x 14 columns
df_method = qry("SELECT * FROM method")# 5 methods x 4 columns
df_ascent = qry("SELECT * FROM ascent")# 4'111'877 ascents x 28 columns

CPU times: user 1min 13s, sys: 14.7 s, total: 1min 28s
Wall time: 1min 34s


###### Info about the type of values in each column (no need to run this every time).

In [6]:
# Get Col + info
columns_in_table = pd.DataFrame()
for i in tables.name:
    df_i = qry("PRAGMA TABLE_INFO(" + i + ");")
    df_i['table_name'] = i
    columns_in_table = columns_in_table.append(df_i)
columns_in_table.shape

(68, 7)

In [7]:
columns_in_table

,cid,name,type,notnull,dflt_value,pk,table_name
0,0,id,INTEGER,1,None,1,user
1,1,first_name,VARCHAR,0,None,0,user
2,2,last_name,VARCHAR,0,None,0,user
3,3,city,VARCHAR,0,None,0,user
4,4,country,VARCHAR,0,None,0,user
5,5,sex,INTEGER,0,None,0,user
6,6,height,INTEGER,0,None,0,user
7,7,weight,INTEGER,0,None,0,user
8,8,started,INTEGER,0,None,0,user
9,9,competitions,VARCHAR,0,None,0,user


&&&&&&&&&&&&&&&&&&&&&&&&&

In [6]:
df_user.columns

Index(['id', 'first_name', 'last_name', 'city', 'country', 'sex', 'height',
       'weight', 'started', 'competitions', 'occupation', 'sponsor1',
       'sponsor2', 'sponsor3', 'best_area', 'worst_area', 'guide_area',
       'interests', 'birth', 'presentation', 'deactivated', 'anonymous'],
      dtype='object')

In [7]:
df_grade.columns

Index(['id', 'score', 'fra_routes', 'fra_routes_input', 'fra_routes_selector',
       'fra_boulders', 'fra_boulders_input', 'fra_boulders_selector',
       'usa_routes', 'usa_routes_input', 'usa_routes_selector', 'usa_boulders',
       'usa_boulders_input', 'usa_boulders_selector'],
      dtype='object')

In [8]:
df_method

,id,score,shorthand,name
0,1,0,redpoint,Redpoint
1,2,53,flash,Flash
2,3,145,onsight,Onsight
3,4,-52,toprope,Toprope
4,5,95,onsight,Onsight


In [9]:
df_ascent.columns

Index(['id', 'user_id', 'grade_id', 'notes', 'raw_notes', 'method_id',
       'climb_type', 'total_score', 'date', 'year', 'last_year', 'rec_date',
       'project_ascent_date', 'name', 'crag_id', 'crag', 'sector_id', 'sector',
       'country', 'comment', 'rating', 'description', 'yellow_id', 'climb_try',
       'repeat', 'exclude_from_ranking', 'user_recommended', 'chipped'],
      dtype='object')

<b> climb type: 0 = sport, 1 = boulder </b>

### Extracting active climbers

We extract the "active" climbers, i.e the users with recorded ascents, and define a dict called "logbook_active_users", whose keys are the active users, and values their ascents. <br/>

In [6]:
%%time
logbook = {} # First we cook up a "logbook" dict, with keys the climbers (users) and values the routes they sent.
for user in df_user['id']:
    logbook[user] = []
for route in df_ascent.itertuples():
    if getattr(route, 'user_id') in logbook:
        logbook[getattr(route, 'user_id')].append(getattr(route, 'id'))
    else:
        logbook[getattr(route, 'user_id')] = [getattr(route, 'id')]

CPU times: user 14.2 s, sys: 3.41 s, total: 17.6 s
Wall time: 18.5 s


In [7]:
print("There are") 
print(len(logbook),'-',df_user.shape[0],'=',len(logbook) - df_user.shape[0], 'users') 
print("""in the ascent database which do not correspond to anyone in the user database!! 
We have added these three mysterious climbers in the logbook dict""")

There are
62596 - 62593 = 3 users
in the ascent database which do not correspond to anyone in the user database!! 
We have added these three mysterious climbers in the logbook dict


In [8]:
logbook_active_users = {} # Getting rid in logbook of climbers with no recorded ascents
for user in logbook:
    if logbook[user] != []:
        logbook_active_users[user] = logbook[user]
print('There are', len(logbook_active_users), 'active climbers in the users database')

There are 36034 active climbers in the users database


In [9]:
active_user = [user for user in range(df_user.shape[0]) if df_user['id'][user] in logbook_active_users]
df_active_user = df_user.loc[active_user].set_index('id') # The dataframe of active users
df_active_user.shape

(36031, 21)

In [10]:
df_active_user.head()

,first_name,last_name,city,country,sex,height,weight,started,competitions,occupation,...,sponsor2,sponsor3,best_area,worst_area,guide_area,interests,birth,presentation,deactivated,anonymous
id,,,,,,,,,,,,,,,,,,,,,
1,first,last,Göteborg,SWE,0,177,73,1996,,,...,,,"Railay beach, Krabi, Thailand",None,,,1976-03-10,,0,0
2,first,last,stockholm,SWE,0,0,0,2000,,,...,,,,,,,None,,0,0
3,first,last,Umeå,SWE,0,180,78,1995,,,...,,,Hell,Umeå,,,1973-09-09,,0,0
4,first,last,Goteborg,SWE,1,165,58,2001,,,...,,,,None,,,1984-07-26,,0,0
5,first,last,North Attleboro,USA,0,0,0,1991,,,...,,,,None,,,1969-05-07,,0,0


In [11]:
%%time
#Set the column 'id' to be the index. 
df_ascent_id = df_ascent.set_index('id')

CPU times: user 1.34 s, sys: 1.74 s, total: 3.09 s
Wall time: 4.1 s


## Average number of years to climb a grade

###### Years of ascents

In [12]:
# No funny things in the 'date' column of the ascent. It's either a unix timestamp or 0.
df_ascent['date'][(df_ascent['date'] !=0)].min()

315529200

In [13]:
pd.to_datetime(315529200, unit = 's').year

1979

In [14]:
# I'm using the column 'date' for the ascent, because the column 'year' has a lot of data
# which makes absolutely no sense at all. 
def send_year(ascent):
    """
    Given an ascent id, returns the year.
    """
    date = pd.to_datetime(df_ascent_id.loc[ascent, 'date'], unit = 's')
    if date == pd.to_datetime(0, unit='s'):
        return 0
    else: 
        return date.year

In [15]:
send_year(411123)

2006

In [16]:
# Some non-sensical starting years, but at least it's 0 or a year number. 
user_started = df_active_user['started']
user_started[(user_started != 0)].min()

1901

###### Birth years; starting climbing years

In [12]:
%%time
#Year or birth of active climbers
birthyears = pd.to_datetime(df_active_user['birth'],errors = 'coerce')
for climber in birthyears.index:
    if birthyears[climber] == None:
        birthyears[climber] = np.nan
    else:
        b_year = birthyears[climber].year
        birthyears[climber] = b_year
#Years active users started climbing
startedyears = df_active_user['started'].replace(0, np.nan)

CPU times: user 1.53 s, sys: 47.3 ms, total: 1.58 s
Wall time: 1.85 s


&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&

###### Age admissible active users

Some users have no birth date, some have no 'started' years, and more annoyingly, some have written down a starting year lower than their birth year!
<br>
We compute how many active climbers have a starting year lower than birth year.  

In [20]:
birthyear = pd.to_datetime(df_active_user['birth'], errors = 'coerce')
birth_year = pd.Series(np.array([birth.year for birth in birthyear]), index=df_active_user.index)

In [21]:
#Replace 0 by nan in 'started'
started_year = df_active_user['started'].replace(0, np.nan)

In [22]:
admissible = np.array([not value for value in (birth_year > started_year)])

In [23]:
print('There are only',admissible.shape[0] - admissible.sum(), 'climbers with starting year lower than their birth year' )

There are only 67 climbers with starting year lower than their birth year


&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&

###### Grade ID

In [13]:
def grade_id(grade):
    """
    Given a grade (given as a string), returns its id number. 
    """
    row = 0
    exit = False 
    while not exit:
        line = df_grade.iloc[row].values
        if grade in line:
            exit = True 
            return df_grade.iloc[row]['id']
        else: row = row +1

###### Average number of years to climb a grade

In [14]:
def achieve_grade(sample, grade, boulder = False):
    """
    Given a grade and a sample of active users, returns the average age and average number of climbing 
    years after which said grade was achieved. 
    Input:
    sample: A sub-dataframe of active users.
    grade: string
    boulder: boolean. False is sport, True is boulder (default = False)
    Output: (2,) array.  
    """
    result = []
    gradeid = grade_id(grade)
    for climber in sample.index:
        mask = (df_ascent_id['user_id'] == climber) & (df_ascent_id['grade_id'] == gradeid) & (df_ascent_id['climb_type'] == boulder)
        climber_sends = df_ascent_id[mask]
        years_of_sending_grade = pd.to_datetime(climber_sends['date'], unit = 's')
        first_year = years_of_sending_grade.min().year
        if first_year == 1970 or np.isnan(first_year):
            #If date is 0, the corresponding year is 1970; 
            #if climber_sends is an empty Serie, first_year is a nan. 
            (age, nb_years) = (np.nan, np.nan)
        else:
            (age, nb_years) = (first_year - birthyears[climber], first_year - startedyears[climber])
        result.append([age, nb_years])
            
    return np.nanmean(np.array(result), axis = 0) #np.nanmean is mean ignoring the nan's

In [15]:
def achieve_grades(sample, grades, boulder = False):
    """
    Given a list of grades and a sample of active users, returns an np.array corresponding to the following:
    Every row corresponds to a grade; first column is average age; second column is average number of climbing years.
    """
    result = achieve_grade(sample, grades[0], boulder)
    for grade in grades[1:]:
        new_line = achieve_grade(sample, grade, boulder)
        result = np.vstack((result, new_line))
        
    return result

In [16]:
usa_grades = [grade for grade in df_grade.loc[34:77,'usa_routes'] if grade != '']

In [17]:
sample = df_active_user.sample(n=500)

In [19]:
%%time
average_achieve_grade = achieve_grades(sample, usa_grades)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:26: RuntimeWarning: Mean of empty slice


CPU times: user 5min 55s, sys: 2min 6s, total: 8min 2s
Wall time: 8min 6s


In [18]:
plt.rcParams['figure.figsize'] = [30, 10]
plt.scatter(usa_grades, average_achieve_grade[:,0])
plt.scatter(usa_grades, average_achieve_grade[:,1])
plt.legend(['age', 'nb of years'], loc = 2)

KeyboardInterrupt: 

Random sample of 500 users (out of 36031 active users). 

&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&& <br>
&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&

In [ ]:
def achieve_grade(grade, boulder = False):
    """
    Given a grade, returns the following 2-dimensional array: every row corresponds to an active climber,
    the first column is the age at which grade was first achieved, second column the number of climbing
    years after which said grade was achieved. 
    Input:
    grade: string
    boulder: boolean. False is sport, True is boulder (default = False)
    Output: (-1,2) array.  
    """
    result = []
    gradeid = grade_id(grade)
    for climber in df_active_user.index:
        birth, started = birth_started(climber)
        mask = (df_ascent_id['user_id'] == climber) & (df_ascent_id['grade_id'] == gradeid) & (df_ascent_id['climb_type'] == boulder)
        climber_sends = df_ascent_id[mask]
        years_of_sending_grade = pd.to_datetime(climber_sends['date'], unit = 's')
        first_year = years_of_sending_grade.min().year
        if first_year == 1970 or np.isnan(first_year):
            #If date is 0, the corresponding year is 1970; 
            #if climber_sends is an empty Serie, first_year is a nan. 
            (age, nb_years) = (np.nan, np.nan)
        else:
            (age, nb_years) = (first_year - birth, first_year - started)
        result.append([age, nb_years])
            
    return np.array(result)

In [43]:
mask = (df_ascent_id['user_id'] == 1) & (df_ascent_id['grade_id'] == 55) & (df_ascent_id['climb_type'] == 0)
climber_sends = df_ascent_id[mask]

In [46]:
years_of_sending_grade = pd.to_datetime(climber_sends['date'], unit = 's')
years_of_sending_grade

id
95      1999-04-05 22:00:00
101     2000-05-08 22:00:00
103     2000-05-24 22:00:00
112     2000-06-21 22:00:00
113     2000-07-01 22:00:00
114     2000-07-14 22:00:00
117     2000-08-06 22:00:00
125     2000-08-24 22:00:00
152     2000-09-15 22:00:00
24338   2001-09-06 22:00:00
32850   2001-07-31 22:00:00
32851   2001-07-31 22:00:00
32852   2001-07-31 22:00:00
Name: date, dtype: datetime64[ns]

In [54]:
first_year = years_of_sending_grade.min().year
first_year

1999

In [45]:
pd.to_datetime(923349600, unit ='s')

Timestamp('1999-04-05 22:00:00')